In [2]:
from lib.opt_types import *
from lib.utils import *
from lib.part_one import f, f_full_data, x_zero

# Stochastic gradient methods - 30 Points


In this problem, you will implement three different versions of stochastic gradient descent to solve the logistic regression problem on the entire NBA dataset.

In order to use the stochastic gradient descent methods, we recast our estimation problem [3](#mjx-eqn-eq3) as follows,

\begin{equation}
\tag{6}
f(\mathbf{x}) = \frac{1}{n}\sum_{i=1}^n\bigg\{\underbrace{\log(1 + \exp(- b_i \mathbf{a}_i^T\mathbf{x})) + \frac{\mu}{2}\|\mathbf{x}\|^2}_{f_i(\mathbf{x})}\bigg\},
\end{equation}

where we for notational convenience suppress the dependency on $\mu$.
As we saw in Problem 1, we have that

\begin{equation}
\nabla f_i(\mathbf{x}) = -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i + \mu \mathbf{x}.
\end{equation}

The objective function can be written as a sum of $n$ terms. We augment the `Function` type introduced in previous notebooks to include the following attributes:

- To access the gradient of the `j`-th term at a point `x` you can write `f.i_grad(j, x)`.
- The number of terms $n$ is stored in the attribute `f.n`. 

Consider the following stochastic gradient update: At the iteration $k$, pick $i_k\in\{1, \ldots, n\}$ uniformly at random and define

\begin{equation}
\mathbf{x}^{k+1} :=  \mathbf{x}^k - \alpha_k\nabla{f}_{i_k}(\mathbf{x}^k).\tag{SGD}
\end{equation}

__(a)__ (2 points) Show that $\nabla f_{i_k}(\mathbf{x})$ is an unbiased estimation of $\nabla f(\mathbf{x})$. Explain why $\nabla f_{i_k}$ is Lipschitz continuous with $L(f_{i_k}) = \|\mathbf{a}_{i_k}\|^2+ \mu $. 

__Hint__: Recall how we computed $L$ in Problem 1. In the following, we will set $L_{\max} = \max_{i\in\{1, \ldots, n\}}L(f_i)$.

ANSWER : To show that $\nabla f_{i_k}(\mathbf{x})$ is an unbiased estimation of $\nabla f(\mathbf{x})$, we need to demonstrate that the expected value of $\nabla f_{i_k}(\mathbf{x})$ is equal to $\nabla f(\mathbf{x})$. The gradient of the objective function $f(\mathbf{x})$ is given by:

$$
\nabla f(\mathbf{x}) = \frac{1}{n} \sum_{i=1}^n \nabla f_i(\mathbf{x}).
$$

Since $i_k$ is chosen uniformly at random from $\{1, \ldots, n\}$, the expected value of $\nabla f_{i_k}(\mathbf{x})$ is:

$$
\mathbb{E}[\nabla f_{i_k}(\mathbf{x})] = \frac{1}{n} \sum_{i=1}^n \nabla f_i(\mathbf{x}).
$$

Therefore, we have:

$$
\mathbb{E}[\nabla f_{i_k}(\mathbf{x})] = \nabla f(\mathbf{x}).
$$

To show that $\nabla f_{i_k}$ is Lipschitz continuous with $L(f_{i_k}) = \|\mathbf{a}_{i_k}\|^2 + \mu$, we need to demonstrate that the gradient $\nabla f_{i_k}(\mathbf{x})$ satisfies the Lipschitz condition. 

We take the gradient of $f_i(\mathbf{x})$ and the Hessian as we already did in part 1:

$$
\nabla f_i(\mathbf{x}) = -b_i \sigma(-b_i \cdot \mathbf{a}_i^T \mathbf{x}) \mathbf{a}_i + \mu \mathbf{x}.
$$

$$
\nabla^2 f_i(\mathbf{x}) = \sigma(-b_i \cdot \mathbf{a}_i^T \mathbf{x})(1 - \sigma(-b_i \cdot \mathbf{a}_i^T \mathbf{x})) \mathbf{a}_i \mathbf{a}_i^T + \mu \mathbf{I}.
$$

The maximum eigenvalue of $\nabla^2 f_i(\mathbf{x})$ provides the Lipschitz constant for $\nabla f_i(\mathbf{x})$, which we already did in Part1, questions j. Therefore, $\nabla f_{i_k}$ is Lipschitz continuous with $L(f_{i_k}) = \|\mathbf{a}_{i_k}\|^2 + \mu$.

__(b)__ (2 points) 
We can use the standard stochastic gradient descent method [SGD](#mjx-eqn-eqSGD) to solve [6](#mjx-eqn-eq6). 
Implement `SGD` by completing the following cells with $\alpha_k =\frac{0.01}{k}$.

**Hint**: For some `N`, the function `np.random.choice(N)` generates a random integer in $\{0, \dots, N-1 \}$.


In [3]:
@dataclass
class SGD_state(OptState):
    x_k: Vector
    k: int

In [4]:
def SGD_update(f, state):
    x_k, k = state
    
    i_k = np.random.choice(f.n)
    
    next_x_k = x_k - (0.01/k)*f.i_grad(i_k, x_k)
    
    return SGD_state(next_x_k, k+1)

def SGD_initialize(f, x_zero):
    return SGD_state(x_zero, 1.0)

In [5]:
SGD = OptAlgorithm(name="SGD", init_state=SGD_initialize, state_update=SGD_update)

__(c)__ (6 points) 
Consider the following stochastic averaging gradient method [SAG](#mjx-eqn-eqSAG) to solve [6](#mjx-eqn-eq6):
\begin{equation}
\begin{cases}
\text{pick } i_k\in\{1, \ldots, n\} \text{ uniformly at random}\\
\mathbf{x}^{k+1} := \mathbf{x}^k - \frac{\alpha_k}{n}\sum_{i=1}^n\mathbf{v}_i^k,
\end{cases}
\end{equation}
where 
\begin{equation}
\mathbf{v}_i^k =
\begin{cases}
\nabla f_i(\mathbf{x}^k) &\text{if}\, i = i_k,\\
\mathbf{v}_i^{k-1} &\text{otherwise}.
\end{cases}
\end{equation}

Implement `SAG` by completing the following cells using the step-size 
$\alpha_k=\frac{0.01}{L_{\max}}$ and $\mathbf{v}^0=\mathbf{0}$. (Note that you can access $L_{\max}$ by writing `f.L_max`.

In [31]:
@dataclass
class SAG_state(OptState):
    x_k: Vector
    v_k: List[Vector]
    alpha_k: float

In [14]:
def SAG_update(f, state):
    x_k, v_k, alpha_k = state

    i_k = np.random.choice(f.n)
    v_k[i_k] = f.i_grad(i_k, x_k)
    avg_grad = np.mean(v_k, axis=0)
    next_x_k = x_k - alpha_k * avg_grad

    return SAG_state(next_x_k, v_k, alpha_k)

def SAG_initialize(f, x_zero):
    v_zero = [np.zeros_like(x_zero) for _ in range(f.n)]
    alpha_k = 0.01 / f.L_max
    return SAG_state(x_zero, v_zero, alpha_k)

In [29]:
SAG = OptAlgorithm(name="SAG", init_state=SAG_initialize, state_update=SAG_update)

In [ ]:
plot_epochs([SAG], f, x_zero, 500000)

__(d)__ (6 points) 
We can improve the convergence rate of SGD by periodically computing the full gradient. `SVRG` uses the following variance reduction scheme:
\begin{equation}
\begin{cases}
\text{if } k = 0 \text{ mod } q:\\
\quad \mathbf{z} = \mathbf{x}^k\\
\quad \tilde{\mathbf{v}} = \nabla f(\mathbf{x}^k)\\
\text{Pick } i_k\in \{1, \ldots, n\} \text{ uniformly at random}\\
\mathbf{d}^k = \nabla f_{i_k}({\mathbf{x}}^k) - \nabla f_{i_k}(\mathbf{z}) + \tilde{\mathbf{v}}\\
{\mathbf{x}}^{k+1} := {\mathbf{x}}^k - \gamma\mathbf{d}^k\\
\end{cases}
\end{equation}

Implement `SVRG` by completing the following cells and fixing the following constant:
$\gamma = 1/L_{\max}$ and $q = 100$.

In [38]:
@dataclass
class SVRG_state(OptState):
    x_k: Vector 
    z: Vector
    tilde_v: Vector
    q: int
    gamma: float
    k: int

In [39]:
def SVRG_update(f, state):
    x_k, z, tilde_v, q, gamma, k = state

    if k % q == 0:
        z = x_k
        tilde_v = f.grad(x_k)

    i_k = np.random.choice(f.n)
    d_k = f.i_grad(i_k, x_k) - f.i_grad(i_k, z) + tilde_v
    next_x_k = x_k - gamma * d_k

    return SVRG_state(next_x_k, z, tilde_v, q, gamma, k + 1)

def SVRG_initialize(f, x_zero):
    return SVRG_state(x_zero, x_zero, np.zeros_like(x_zero), 100, 1/f.L_max, 0)

In [40]:
SVRG = OptAlgorithm(name="SVRG", init_state=SVRG_initialize, state_update=SVRG_update)

__(e)__ (8 points) Another variance reduction method is `SARAH`. The scheme can be described as follows:

\begin{equation}
\begin{cases}
\text{if } k = 0 \text{ mod } q:\\
\quad {\mathbf{x}}^k = {\mathbf{z}}\\
\quad {\mathbf{v}}^k = \nabla f({\mathbf{z}})\\
\quad t \in \{0, \ldots, q-1\} \text{ uniformly at random}\\
\text{if } k = t \text{ mod } q:\\
\quad {\mathbf{z}} = {\mathbf{x}}^{k}\\
\text{Pick } i_k\in \{1, \ldots, n\} \text{ uniformly at random}\\
{\mathbf{v}}^{k+1} = \nabla f_{i_k}({\mathbf{x}}^k) - \nabla f_{i_k}({\mathbf{x}^{k-1}}) + {\mathbf{v}}^{k}\\
{\mathbf{x}}^{k+1} := {\mathbf{x}}^k - \gamma {\mathbf{v}}^{k+1}\\
\end{cases}
\end{equation}


Implement `SARAH` by completing the following cells. Pick `q=100` and $\gamma = \frac{1}{L_{\max}}$

In [35]:
@dataclass
class SARAH_state(OptState):
    x_k: Vector 
    prev_x_k: Vector
    v_k: Vector
    z: Vector
    q: int
    t: int
    gamma: float
    k: int

In [36]:
def SARAH_update(f, state):
    x_k, prev_x_k, v_k, z, q, t, gamma, k = state

    if k % q == 0:
        x_k = z
        v_k = f.grad(z)
        t = np.random.choice(q)

    if k % q == t:
        z = x_k

    i_k = np.random.choice(f.n)
    v_k_next = f.i_grad(i_k, x_k) - f.i_grad(i_k, prev_x_k) + v_k
    next_x_k = x_k - gamma * v_k_next

    return SARAH_state(next_x_k, x_k, v_k_next, z, q, t, gamma, k + 1)

def SARAH_initialize(f, x_zero):
    return SARAH_state(x_zero, x_zero, np.zeros_like(x_zero), x_zero, 100, np.random.choice(100), 1 / f.L_max, 0)

In [37]:
SARAH = OptAlgorithm(name="SARAH", init_state=SARAH_initialize, state_update=SARAH_update)

__(f)__ (6 points) Another variance reduction method is `Spider`. The scheme can be described as follows:

\begin{equation}
\begin{cases}
\text{if } k = 0 \text{ mod } q:\\
\quad {\mathbf{v}}^k = \nabla f({\mathbf{x}}^k)\\
\text{if } k \neq 0 \text{ mod } q:\\
\quad \text{Pick } i_k\in \{1, \ldots, n\} \text{ uniformly at random}\\
\quad {\mathbf{v}}^k = \nabla f_{i_k}({\mathbf{x}}^k) - \nabla f_{i_k}({\mathbf{x}}^{k-1}) + {\mathbf{v}}^{k-1}\\
\eta := \min \big(\frac{1}{\left \| {\mathbf{v}}^{k} \right \|_2}, \frac{1}{\epsilon}\big)\\
{\mathbf{x}}^{k+1} := {\mathbf{x}}^k - \gamma \eta {\mathbf{v}}^{k}\\
\end{cases}
\end{equation}


Implement `Spider` by completing the following cells. Pick `q=100`, $\epsilon = 0.05$ and $\gamma = \frac{1}{L_{\max}}$.

In [42]:
@dataclass
class Spider_state(OptState):
    x_k: Vector 
    prev_x_k: Vector
    prev_v_k: Vector
    q: int
    gamma: float
    k: int
    epsilon: float

In [43]:
import numpy
from numpy.linalg import norm

def Spider_update(f, state):
    x_k, prev_x_k, prev_v_k , gamma, q, epsilon, k = state

    if k % q == 0:
        v_k = f.grad(x_k)
    else:
        i_k = np.random.randint(f.n)
        v_k = f.i_grad(i_k, x_k) - f.i_grad(i_k, prev_x_k) + prev_v_k
    
    eta = min(1 / norm(v_k, 2), 1 / epsilon)
    next_x_k = x_k - gamma * eta * v_k

    return Spider_state(x_k=next_x_k, prev_x_k=x_k, prev_v_k=v_k, q=q, gamma=gamma, k=k+1, epsilon=epsilon)

def Spider_initialize(f, x_zero):
    return Spider_state(x_k=x_zero, prev_x_k=x_zero, prev_v_k=f.grad(x_zero), q=100, gamma=1 / f.L_max, k=0, epsilon=0.05)

In [44]:
Spider = OptAlgorithm(name="Spider", init_state=Spider_initialize, state_update=Spider_update)

In [ ]:
plot_epochs([SVRG, SARAH, Spider], f_full_data, x_zero, 50000)